In [2]:
%pip install sagemaker --upgrade
import sagemaker

#assert sagemaker.__version__ >= "2.75.0"



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [27]:
!apt-get update
!apt-get install sudo


Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://security.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://security.debian.org/debian-security bullseye-security/main amd64 Packages [265 kB]
Get:4 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:5 http://deb.debian.org/debian bullseye/main amd64 Packages [8062 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [18.8 kB]
Fetched 8554 kB in 2s (5509 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  sudo
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 1061 kB of archives.
After this operation, 4699 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian bullseye/main amd64 sudo amd64 1.9.5p2-3+deb11u1 [1061 kB]
Fetched 1061 kB in 0s (23.5 MB/s)
debconf: delaying 

In [28]:
# For notebook instances (Amazon Linux)
!sudo yum update -y
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs git -y
#For other environments (Ubuntu)
!sudo apt-get update -y
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs git -y


sudo: yum: command not found
Detected operating system as debian/11.
Checking for curl...
Installing curl...
main: line 23: yum: command not found
main: line 165: /etc/yum.repos.d/github_git-lfs.repo: No such file or directory

Unable to run: 
    curl https://packagecloud.io/install/repositories/github/git-lfs/config_file.repo?os=debian&dist=11&source=script

Double check your curl installation and try again.
sudo: yum: command not found
Hit:1 http://security.debian.org/debian-security bullseye-security InRelease
Hit:2 http://deb.debian.org/debian bullseye InRelease
Hit:3 http://deb.debian.org/debian bullseye-updates InRelease
Reading package lists... Done
Detected operating system as debian/11.
Checking for curl...
Installing curl...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libcurl4
The following NEW packages will be installed:
  curl libcurl4
0 upgraded, 2 newly installed, 0 to remove and

In [2]:
# Install the required packages
%pip install "InstructorEmbedding>=1.0.1" "transformers==4.20.0" --upgrade
%pip install "sagemaker>=2.197.0"  --upgrade



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


In [4]:
repository = "hkunlp/instructor-xl"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/custom_inference/{model_id}/model.tar.gz"

In [5]:
!git lfs install
!git clone https://huggingface.co/$repository


Git LFS initialized.
Cloning into 'instructor-xl'...
remote: Enumerating objects: 85, done.
remote: Total 85 (delta 0), reused 0 (delta 0), pack-reused 85
Unpacking objects: 100% (85/85), 625.13 KiB | 223.00 KiB/s, done.


In [ ]:
#!git lfs pull


In [ ]:
import os

def get_folder_sizes(base_folder):
    folder_sizes = {}
    
    for entry in os.scandir(base_folder):
        if entry.is_dir(follow_symlinks=False):
            folder_path = entry.path
            total_size = 0
            for dirpath, dirnames, filenames in os.walk(folder_path):
                for f in filenames:
                    fp = os.path.join(dirpath, f)
                    if not os.path.islink(fp):
                        total_size += os.path.getsize(fp)
            folder_sizes[entry.name] = total_size

    return folder_sizes

# Usage example
base_folder = './'
sizes = get_folder_sizes(base_folder)

# Printing the sizes in a readable format
for folder, size in sizes.items():
    print(f"Size of folder '{folder}': {size / (1024 * 1024):.2f} MB")


In [15]:
# if the model.bin not downloaded
##!git clone https://huggingface.co/instructor-xl/pytorch_model.bin

Cloning into 'pytorch_model.bin'...
Username for 'https://huggingface.co': ^C


In [6]:
!mkdir code

In [7]:
# inference.py is the entrypoint for the container
%%writefile code/inference.py

from InstructorEmbedding import INSTRUCTOR
import torch

def model_fn(model_dir):
    model = INSTRUCTOR(model_dir)
    model.max_seq_length = 768
    return model


def predict_fn(data, model):
    # destruct model and tokenizer
    model = model

    # Tokenize sentences
    sentences = data.pop("inputs", data)
    
    model =  model.encode(sentences)
    
    # return dictonary, which will be json serializable
    return {"vectors": sentence_embeddings.tolist()}


Writing code/inference.py


In [8]:
%%writefile code/requirements.txt

InstructorEmbedding>=1.0.1
transformers==4.20.0
datasets>=2.2.0
jsonlines
numpy
requests>=2.26.0
scikit_learn>=1.0.2
scipy
sentence_transformers>=2.2.0
torch
tqdm
rich

Writing code/requirements.txt


In [9]:
!cp -r code/ $model_id/code/


In [46]:
#%cd ..
%cd $model_id
!tar zcvf model.tar.gz *


/root
/root/instructor-xl
1_Pooling/
1_Pooling/config.json
2_Dense/
2_Dense/pytorch_model.bin
2_Dense/config.json
README.md
code/
code/requirements.txt
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/inference.py
config.json
config_sentence_transformers.json
modules.json
pytorch_model.bin
sentence_bert_config.json
special_tokens_map.json
spiece.model
tokenizer.json
tokenizer_config.json


In [ ]:
!aws s3 cp model.tar.gz $s3_location



In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel


# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_location,       # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.12",  # transformers version used
   pytorch_version="1.9",        # pytorch version used
   py_version='py38',            # python version used
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.4xlarge"
    )


-------------

In [ ]:
data = {
  "inputs": "the mesmerizing performances of the leads keep the film grounded and keep the audience riveted .",
}

res = predictor.predict(data=data)
print(res)
#   {'vectors': [0.005078191868960857, -0.0036594511475414038, .....]}
